# DungeonBreak Narrative Space (Game-Aligned)

This tutorial uses the in-game thematic basis vectors as the only default demo axes.

C++ touchpoints:
- `FVectorND` and basis vectors in `Plugins/NarrativeEngine/Source/NarrativeEngine/Public/NarrativeCoreData.h`
- Runtime simulation in `Plugins/NarrativeEngine/Source/NarrativeEngine/Private/NarrativeSubsystem.cpp`
- Dialog asset type in `Plugins/NarrativeEngine/Source/NarrativeEngine/Public/NarrativeStaticData.h`


In [ ]:
import sys
from pathlib import Path

for base in [Path.cwd(), Path.cwd().parent]:
    if (base / 'src' / 'dungeonbreak_narrative').exists():
        sys.path.insert(0, str(base))
        break

from dungeonbreak_narrative import (
    EVENT_DIALOG,
    EVENT_REST,
    EVENT_TRAINING,
    EVENT_TRAVEL,
    SCENE_HOME_MOTHER,
    SCENE_HOME_MERCHANT,
    SCENE_TOWN_SQUARE,
    apply_one_event,
    available_dialogs,
    initial_state,
    load_game_traits_manifest,
    load_narrative_snapshot,
    run_playthrough,
    validate_game_alignment_warn_only,
)

import numpy as np
import plotly.graph_objects as go


In [ ]:
alignment = validate_game_alignment_warn_only()
print('Alignment OK:', alignment['ok'])
for warning in alignment['warnings']:
    print('WARN:', warning)

traits = load_game_traits_manifest()
snapshot = load_narrative_snapshot()
print('Trait count:', len(traits))
print('Traits:', ', '.join(traits))
print('Snapshot entities:', len(snapshot.get('entities', [])))
print('Snapshot dialogs:', len(snapshot.get('dialogs', [])))


## Event Simulation

This run consumes manifest traits plus snapshot-defined forces/locations.
If snapshot force/location maps are partial, the engine emits warnings instead of fabricating values.


In [ ]:
state = initial_state(scene_id=SCENE_TOWN_SQUARE, entity_name='Kaiza', snapshot=snapshot)

# Use snapshot dialog labels only (no invented labels).
dialog_labels = [
    d.get('label')
    for d in snapshot.get('dialogs', [])
    if d.get('label')
]

events = [
    {'type': EVENT_TRAINING, 'scene_id': SCENE_TOWN_SQUARE},
    {'type': EVENT_TRAVEL, 'scene_id': SCENE_HOME_MOTHER},
    {'type': EVENT_REST, 'scene_id': SCENE_HOME_MOTHER},
]
if dialog_labels:
    events.append({'type': EVENT_DIALOG, 'scene_id': SCENE_HOME_MOTHER, 'chosen_dialog': dialog_labels[0]})

events.append({'type': EVENT_TRAVEL, 'scene_id': SCENE_HOME_MERCHANT})

event_log = run_playthrough(events, start_state=state, snapshot=snapshot)

rows = []
for row in event_log:
    rows.append({
        'event_index': row['event_index'],
        'event_type': row['event_type'],
        'scene_id': row['scene_id'],
        'level': row['level'],
        'effort_pool': round(row['effort_pool'], 3),
        'chosen_dialog': row['chosen_dialog'],
        'force_applied': row['force_applied'],
        'warnings': '; '.join(row['warnings'][-2:]) if row['warnings'] else '',
    })

for entry in rows:
    print(entry)


## 10D Trait Trajectory (2D Slices)


In [ ]:
if not event_log:
    print('No events available.')
else:
    vectors = np.array([row['vector'] for row in event_log], dtype=float)
    variances = vectors.var(axis=0)
    ranked = np.argsort(-variances)

    if len(traits) < 2:
        print('Need at least two traits for 2D projection.')
    else:
        idx_a = int(ranked[0])
        idx_b = int(ranked[1]) if len(traits) > 1 else 0

        fig = go.Figure()
        fig.add_trace(
            go.Scatter(
                x=vectors[:, idx_a],
                y=vectors[:, idx_b],
                mode='lines+markers+text',
                text=[str(r['event_index']) for r in event_log],
                textposition='top center',
                name='Kaiza path',
            )
        )
        fig.update_layout(
            title=f'Game-Aligned Trait Slice: {traits[idx_a]} vs {traits[idx_b]}',
            xaxis_title=traits[idx_a],
            yaxis_title=traits[idx_b],
        )
        fig.show()

        if np.allclose(variances, 0.0):
            print('All trait variances are 0. Snapshot appears partial or force maps are missing.')


## Dialog Availability from Snapshot Locations


In [ ]:
if not event_log:
    print('No event log.')
else:
    for row in event_log:
        options = available_dialogs(
            row['vector'],
            scene_id=row['scene_id'],
            snapshot=snapshot,
            trait_names=traits,
        )
        labels = [label for label, _, _ in options]
        print(f"event {row['event_index']:>2} @ {row['scene_id']:<14} -> {labels if labels else 'no location-backed options'}")
